# Analyzing a large dataset on the IP2I SLURM Farm

## The New York City taxi dataset

In [1]:
import pprint
import pathlib

taxis = pathlib.Path("/gridgroup/cms/cbernet/test_data/nyc_taxis")

t2011 = sorted(list((taxis / "2011").glob("*.csv")))
pprint.pprint(t2011)

[PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-01.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-02.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-03.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-04.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-05.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-06.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-07.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-08.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-09.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-10.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2011/yellow_tripdata_2011-11.csv'),

In [2]:
[f.stat().st_size / 1024**3 for f in t2011]

[2.3046080265194178,
 2.4255120931193233,
 2.746128797531128,
 2.5176544673740864,
 2.6636097356677055,
 2.5855863811448216,
 2.522386613301933,
 2.2658982882276177,
 2.497904699295759,
 2.6866614799946547,
 2.4851463064551353,
 2.5523328203707933]

## Set up the dask cluster

In [3]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # queue='normal',
    project="cms",
    cores=8,
    memory="64 GB", 
    walltime="02:00:00",
)

/home/cms/cbernet/miniconda3/envs/dask2/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
cluster.scale(jobs=8) 

In [5]:
from dask.distributed import Client
client = Client(cluster)

We now have a cluster with 8x8=64 workers, and in total 512 GB of RAM. This is enough to hold the entire taxi dataset in the cluster memory. 

Dask comes with a handy dashboard:

In [6]:
cluster.dashboard_link

'http://134.158.83.2:8787/status'

In another browser tab, connect to the dashboad. And if you're doing ssh tunnelling, connect to http://localhost:8787, or whatever port you've chosen in your port mapping. 

## Dask dataframe : A first look at the data

In [7]:
import dask
import dask.dataframe as dd

In [8]:
df = dd.read_csv(
    taxis / "2011/*.csv", 
    dtype={"tip_amount": "float64", "tolls_amount": "float64"}   # data not clean, need to cast type
)

Let's have a look at the first rows of the dataframe. This is a fast operation, as there is no need to process the full dataframe to get these rows: 

In [10]:
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2011-01-29 02:38:35,2011-01-29 02:47:07,1,1.2,-74.005254,40.729084,1,N,-73.988697,40.727127,CSH,6.1,0.5,0.5,0.0,0.0,7.1
1,CMT,2011-01-28 10:38:19,2011-01-28 10:42:18,1,0.4,-73.968585,40.759171,1,N,-73.964336,40.764665,CSH,4.1,0.0,0.5,0.0,0.0,4.6
2,CMT,2011-01-28 23:49:58,2011-01-28 23:57:44,3,1.2,-73.980710,40.742390,1,N,-73.987028,40.729532,CSH,6.1,0.5,0.5,0.0,0.0,7.1
3,CMT,2011-01-28 23:52:09,2011-01-28 23:59:21,3,0.8,-73.993773,40.747329,1,N,-73.991378,40.750050,CSH,5.3,0.5,0.5,0.0,0.0,6.3
4,CMT,2011-01-28 10:34:39,2011-01-28 11:25:50,1,5.3,-73.991475,40.749936,1,N,-73.950237,40.775626,CSH,25.3,0.0,0.5,0.0,0.0,25.8


Now, we check the dataframe shape

In [11]:
df.shape

(Delayed('int-d198c96a-0b5a-4c0e-b8e8-61d84aa680e8'), 18)

The last number is the number of columns. 

The first number is the number of rows. To get it, we need to scan the entire dataframe. So let's compute this number on the cluster (execute the following cell and go check your dashboard :)

In [13]:
n_trips = df.shape[0].compute()
n_trips 

176897199

There are 176 million taxi trips in our dataset! 

This is a large text dataset, that cannot be processed easily on a single machine. That's why we use a cluster. 

For first tests, a good practice is to select a sample of this dataset that can fit in the memory of the local machine. Here, we sample with a probability of `1e-5` to get about 2000 taxi trips (check your dashboard :)

In [14]:
sample = df.sample(frac=1e-5)
type(sample)

dask.dataframe.core.DataFrame

Before compute, we have a small dask dataframe. We compute it to turn it into a pandas dataframe:

In [15]:
sample = sample.compute()
print(type(sample))
print(sample.shape)

<class 'pandas.core.frame.DataFrame'>
(1875, 18)


Now, we can use our small sample directly, e.g.: 

In [16]:
sample["tip_amount"].describe()

count    1875.000000
mean        0.995941
std         1.616802
min         0.000000
25%         0.000000
50%         0.000000
75%         1.800000
max        20.000000
Name: tip_amount, dtype: float64

In [17]:
sample.describe()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
count,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000
mean,1.641067,2.801221,-71.967649,39.663557,1.028267,-72.044399,39.684148,10.308693,0.338933,0.498133,0.995941,0.205013,12.346715
std,1.244719,3.278656,12.027601,6.567075,0.243925,11.805137,6.502751,8.131800,0.369500,0.030502,1.616802,1.031700,9.689979
min,0.000000,0.000000,-75.425293,0.000000,0.000000,-75.414491,0.000000,2.500000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,1.000000,1.045000,-73.992424,40.734685,1.000000,-73.991006,40.732678,5.700000,0.000000,0.500000,0.000000,0.000000,7.200000
50%,1.000000,1.800000,-73.981693,40.753315,1.000000,-73.979859,40.753418,8.100000,0.500000,0.500000,0.000000,0.000000,9.600000
75%,2.000000,3.110000,-73.966687,40.767927,1.000000,-73.964302,40.768134,11.700000,0.500000,0.500000,1.800000,0.000000,13.680000
max,6.000000,41.400000,0.000000,41.027519,5.000000,0.000000,41.027519,138.000000,1.000000,0.500000,20.000000,15.000000,167.600000


## Parallel analysis : Probability to get a tip

Let's compute the probability to get a tip. 

First, we design our analysis on our sample. It's easy:

In [18]:
(sample["tip_amount"]>0).sum() / sample.shape[0]

0.44106666666666666

We can also write a small function to do this: 

In [19]:
def tip_prob(df): 
    n_tips = (df["tip_amount"]>0).sum()
    n_trips = df.shape[0]
    return n_tips/n_trips

In [20]:
tip_prob(sample)

0.44106666666666666

To run the computation on the whole dataset, we just pass the full dask dataframe to the function: 

In [21]:
result = tip_prob(df)
result

Delayed('_inner-0fc1a8ddccaa79ca505fec6c18a07590')

In [22]:
result.compute()

0.41981193269204903

Terminate your cluster: 

In [23]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


## Conclusion and outlook

At IP2I, we're probably not going to analyze text files very often.

But this can happen. For example, Denis could use Dask to analyse disk space usage on gridgroup or on the storage element. Dask is also well suited to the analysis of JSON data.

And (dask) dataframes are not limited to text data. For example, they can be used to analyse columnar binary data, or images. 

A few inspirational links: 

* [Dask-ML](https://ml.dask.org/): Dask-ML provides scalable machine learning in Python using Dask alongside popular machine learning libraries like Scikit-Learn, XGBoost, and others.
* [Dask-Image](https://examples.dask.org/applications/image-processing.html)
* [Xarray](https://docs.xarray.dev/en/stable/gallery.html): Wraps Dask Array, offering the same scalability, but with axis labels which add convenience when dealing with complex datasets. A good way to deal with very large images
* [Datashader](https://datashader.org/): plot images that are too large to fit in memory. Can be used in conjunction with dask.
* [The Dask ecosystem](https://docs.dask.org/en/latest/ecosystem.html): the full dask ecosystem


